In [ ]:
import requests
import json
import pandas as pd
import time
from config import API_key

# Данные о детских книгаг

In [ ]:
children_data = pd.read_csv("Childen_books_data.csv")

In [ ]:
children_data

,Unnamed: 0,Title,Price,Currency,Rating,Number of reviews,Original Price,Main Genre
0,0,Sundays with Grandma,20.99,$,NaN,NaN,20.99,Children's book
1,1,The Enchanted Post: Letters To Santa Claus,24.99,$,5.0,6.0,24.99,Children's book
2,2,Paces and Cringer - Cat Detectives,7.50,$,5.0,3.0,7.50,Children's book
3,3,Dr. Seuss's Beginner Book Boxed Set Collection...,26.47,$,4.9,24893.0,49.95,Children's book
4,4,The Rainbow Fish,5.04,$,4.8,6797.0,9.95,Children's book
...,...,...,...,...,...,...,...,...
1623,1623,Growly Bear: A Bear Who Discovers It's Best to...,6.99,$,4.5,138.0,6.99,Children's book
1624,1624,"I Love You, My Little Unicorn: A Magical and E...",9.03,$,4.9,228.0,10.99,Children's book
1625,1625,I am a Buddy of L.O.: A Novel,17.99,$,NaN,NaN,17.99,Children's book
1626,1626,What Happens When You CATCH A COLD?: LUKA VS. ...,10.99,$,5.0,3.0,10.99,Children's book


In [ ]:
titles = children_data['Title'].dropna().tolist()

base_url = "https://www.googleapis.com/books/v1/volumes"
results = []

for title in titles:
    search_query = title.replace(' ', '+')
    url = f"{base_url}?q=intitle:{search_query}"
    flag = True
    while flag:
      try:
          response = requests.get(url, headers={'accept': 'application/json', 'X-API-KEY': API_key})
          if response.status_code == 200:
              flag = False
              response_data = response.json()

              if 'items' in response_data and len(response_data['items']) > 0:
                  first_result = response_data['items'][0]['volumeInfo']
                  results.append({
                      "Title": title,
                      "API Title": first_result.get('title'),
                      "Authors": first_result.get('authors', []),
                      "Publisher": first_result.get('publisher'),
                      "Published Date": first_result.get('publishedDate'),
                      "Description": first_result.get('description'),
                      "Page Count": first_result.get('pageCount'),
                      "Print Type": first_result.get('printType'),
                      "Categories": first_result.get('categories', []),
                      "Average Rating": first_result.get('averageRating'),
                      "Ratings Count": first_result.get('ratingsCount'),
                      "Language": first_result.get('language'),
                  })
              else:
                  results.append({
                      "Title": title,
                      "API Title": None,
                      "Authors": None,
                      "Publisher": None,
                      "Published Date": None,
                      "Description": None,
                      "Page Count": None,
                      "Print Type": None,
                      "Categories": None,
                      "Average Rating": None,
                      "Ratings Count": None,
                      "Language": None,
                  })
          else:
              print(f"Ошибка при запросе для '{title}': {response.status_code}")
              time.sleep(1)
      except Exception as e:
          print(f"Ошибка для '{title}': {e}")


    time.sleep(0.2)


Ошибка при запросе для 'Llama Llama I Love You': 429
Ошибка при запросе для 'Who’s Hiding? (1) (My First Baby Books)': 429
Ошибка при запросе для 'Green Eggs and Ham': 429
Ошибка при запросе для 'Around the World: Keo Fights for Freedom — Hong Kong': 429
Ошибка при запросе для 'I Am a Big Brother (Caroline Jayne Church)': 429
Ошибка при запросе для 'The Gruffalo': 429
Ошибка при запросе для 'The sun rises still': 429
Ошибка при запросе для 'Unicorns Love Ice Cream (Mythical Feasts)': 429
Ошибка при запросе для 'Storie della Buonanotte per Bambini: Racconti Fantastici per Dormire Felici e Calmi, con Avventure Dove la Paura Fa Sorridere, tra Coraggio, Amicizia e la Magia della Notte (Italian Edition)': 429
Ошибка при запросе для 'The Very Busy Spider': 429
Ошибка при запросе для 'You're My Little Honey Bunny': 429
Ошибка при запросе для 'Where Hands Go: An Introduction to Safe and Unsafe Touch': 429
Ошибка при запросе для 'Where Hands Go: An Introduction to Safe and Unsafe Touch': 429
Ош

In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv("children_api_results.csv", index=False)


# Все стальные жанры

In [ ]:
all_books = pd.read_csv("All_added_books_data.csv")

In [ ]:
titles = all_books['Title'].dropna().tolist()

base_url = "https://www.googleapis.com/books/v1/volumes"
results = []

for title in titles:
    search_query = title.replace(' ', '+')
    url = f"{base_url}?q=intitle:{search_query}"
    flag = True
    while flag:
      try:
          response = requests.get(url, headers={'accept': 'application/json', 'X-API-KEY': API_key})
          if response.status_code == 200:
              flag = False
              response_data = response.json()

              if 'items' in response_data and len(response_data['items']) > 0:
                  first_result = response_data['items'][0]['volumeInfo']
                  results.append({
                      "Title": title,
                      "API Title": first_result.get('title'),
                      "Authors": first_result.get('authors', []),
                      "Publisher": first_result.get('publisher'),
                      "Published Date": first_result.get('publishedDate'),
                      "Description": first_result.get('description'),
                      "Page Count": first_result.get('pageCount'),
                      "Print Type": first_result.get('printType'),
                      "Categories": first_result.get('categories', []),
                      "Average Rating": first_result.get('averageRating'),
                      "Ratings Count": first_result.get('ratingsCount'),
                      "Language": first_result.get('language'),
                  })
              else:
                  results.append({
                      "Title": title,
                      "API Title": None,
                      "Authors": None,
                      "Publisher": None,
                      "Published Date": None,
                      "Description": None,
                      "Page Count": None,
                      "Print Type": None,
                      "Categories": None,
                      "Average Rating": None,
                      "Ratings Count": None,
                      "Language": None,
                  })
          else:
              print(f"Ошибка при запросе для '{title}': {response.status_code}")
              time.sleep(1)
      except Exception as e:
          print(f"Ошибка для '{title}': {e}")


    time.sleep(0.2)


In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv("all_api_books.csv", index=False)